# Trabajo de Curso de Visión por Computador
## Autores:
- Héctor Wood Santana
- Alejandro Viera Ruiz 

### Notas:
#### Mano Derecha:
    - Índice = ratón
    - Pulgar + Corazón = Click Izquierdo
    - Pulgar + Anular = Click Derecho
    - Pulgar +  Meñique = Click Central??? ¿Flag para cambiar modos y asi tener mas opciones en la mano izquierda?
#### Mano Izquierda:
    - Pulgar + Índice = Abrir Teclado Virtual 
    - Pulgar + Corazón = Macro 1
    - Pulgar + Anular = Macro 2
    - Pulgar + Meñique = Macro 3


### Tareas:
    1º  Hacer Mano derecha (Héctor)
    2º  Pensar en Macros para las 8 opciones de la maño izquierda (Alejandro y ¿Héctor?)
    3º  Buscar como abrir aplicaciones (Alejandro)
    4º  Mirar calculo de la distancia para los clicks relativa al bounding box de la mano (Héctor)
    5º  Hacer Memoria
    6º  ¿Hacer juego?


In [5]:
import cv2
import mediapipe as mp
import pyautogui

# Inicializamos pyautogui
pyautogui.FAILSAFE = True


# Inicializar MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Input Camera
vid = cv2.VideoCapture(0, cv2.CAP_DSHOW)

# Obtenemos la resolución de la pantalla
screen_width, screen_height = pyautogui.size()

# Constantes
width = 960
height = 720

# Bucle de ejecución
while True:
    # Fotograma de la cámara
    ret, frame = vid.read()
    if not ret:
        break

    # Cambiar tamaño del fotograma
    frame = cv2.resize(frame, (width, height))
    # Voltear el fotograma
    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape  
    # Convertir el color BGR a RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Procesar el fotograma
    result = hands.process(rgb_frame)

    # Procesamiento
    if result.multi_hand_landmarks:
        for idx, hand_landmarks in enumerate(result.multi_hand_landmarks):
            # Dibujar los puntos clave de la mano en la imagen
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            # Obtener la información de la mano (derecha o izquierda)
            handedness = result.multi_handedness[idx].classification[0].label
            print(f"Hand {idx + 1} is {handedness}")
    
    # Muestra fotograma
    cv2.imshow('Vid', frame)

    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()




Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 2 is Right
Hand 1 is Right
Hand 1 is Right
Hand 2 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 is Right
Hand 1 i